In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import sys
sys.path.append('..')

import autorobot as ar
from autorobot.constants import (
    RProjType,
)

In [4]:
rb = ar.initialize()

In [5]:
from autorobot.synonyms import synonyms

print(ar.app.synonyms)

{'BUILDING': <RProjType.BUILDING: 19>, 'FRAME_2D': <RProjType.FRAME_2D: 1>, 'FRAME_3D': <RProjType.FRAME_3D: 4>, 'SHELL': <RProjType.SHELL: 7>, 'TRUSS_2D': <RProjType.TRUSS_2D: 2>, 'TRUSS_3D': <RProjType.TRUSS_3D: 5>, 'PERM': <RCaseNature.PERM: 0>, 'IMPOSED': <RCaseNature.IMPOSED: 1>, 'WIND': <RCaseNature.WIND: 2>, 'SNOW': <RCaseNature.SNOW: 3>, 'ACC': <RCaseNature.ACC: 5>, 'SLS': <RCombType.SLS: 1>, 'ULS': <RCombType.ULS: 0>, 'LINEAR': <RAnalysisType.LINEAR: 1>, 'NON_LIN': <RAnalysisType.NON_LIN: 2>, 'COMB_LINEAR': <RAnalysisType.COMB_LINEAR: 0>, 'COMB_NON_LIN': <RAnalysisType.COMB_NON_LIN: -1>, <RCaseNature.PERM: 0>: 0, <RCaseNature.IMPOSED: 1>: 1, <RCaseNature.WIND: 2>: 2, <RCaseNature.SNOW: 3>: 3, <RCaseNature.ACC: 5>: 5, <RProjType.BUILDING: 19>: 19, <RProjType.FRAME_3D: 4>: 4, <RProjType.SHELL: 7>: 7, <RAnalysisType.COMB_NON_LIN: -1>: -1}


In [6]:
rb.new('SHELL')

In [7]:
n1 = rb.nodes.create(0,0,0)
n2 = rb.nodes.create(10,0,0)

In [8]:
bar = rb.bars.create(1, 2)

In [9]:
label = ar.RobotOM.IRobotLabel(bar.GetLabel(ar.constants.RLabelType.MAT))
print(label)

System.__ComObject


In [10]:
case_1 = rb.cases.create_case(1, 'case 1', 'PERM', 'LINEAR', overwrite=True)

In [11]:
case_1.add_bar_udl('1', fz=-2., desc='udl on bar 1')
case_1.add_mass_accel(ax=3., desc='mass acceleration')
case_1.add_self_weight(factor=2., desc='self weight')

AttributeError: ExtendedSimpleCase has not attribute 'add_mass_accel'.

In [ ]:
rec = ar.RobotOM.IRobotLoadRecord(case_1.Records.Create[int](89))
print(dir(rec))

In [ ]:
print(dir(case_1))
case_1.loads

case = rb.cases.get(1)
print(case.loads)

In [ ]:
print(dir(ar.RobotOM.IRobotLoadRecordType))
print(ar.RobotOM.IRobotLoadRecordType.I_LRT_DEAD)
print(ar.RobotOM.IRobotLoadRecordType.I_LRT_BAR_DEAD)

for name in dir(ar.RobotOM.IRobotLoadRecordType):
    if name.startswith('I_LRT'):
        print(name, ': ' ,getattr(ar.RobotOM.IRobotLoadRecordType, name))

In [ ]:
print(dir(ar.RobotOM.IRobotLoadRecordType))

type(ar.RobotOM.IRobotLoadRecordType)

In [ ]:
ar.RobotOM.IRobotLoadRecordType.I_LRT_BODY_FORCE = 89

print(type(ar.RobotOM.IRobotLoadRecordType.I_LRT_DEAD))
print(type(ar.RobotOM.IRobotLoadRecordType.I_LRT_BODY_FORCE))

In [ ]:
import clr
from pathlib import Path

# Searching for ``interop.RobotOM.dll``
p = Path(r'C:\Program Files\Autodesk')

suffix = r'System\EXE\interop.RobotOM.dll'
_robot_dll_path = (
    str(p) if str(p).endswith(suffix)
    else str(next(p.rglob(suffix), ''))
)


clr.AddReference(_robot_dll_path)

clr.setPreload(True)
import RobotOM  # NOQA F401 F402

from RobotOM import IRobotLoadRecordType

IRobotLoadRecordType.I_LRT_BODY_FORCE = 89

print(IRobotLoadRecordType.I_LRT_BODY_FORCE)
rec = ar.RobotOM.IRobotLoadRecord(case_1.Records.Create(ar.RobotOM.IRobotLoadRecordType.I_LRT_DEAD))
rec = ar.RobotOM.IRobotLoadRecord(case_1.Records.Create(IRobotLoadRecordType.I_LRT_BODY_FORCE))
print(dir(rec))

In [12]:
dir(case_1.Records.Create)

['__call__',
 '__class__',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [43]:
import clr
print(clr.ListAssemblies.__doc__)
l = clr.ListAssemblies(False)
print(list(l))
dir(clr)

System.String[] ListAssemblies(Boolean)
['mscorlib', 'clrmodule', 'Python.Runtime', 'System.Core', 'System.Configuration', 'System.Xml', 'System', '__CodeGenerator_Assembly', 'e__NativeCall_Assembly', 'Interop.RobotOM']


['AddReference',
 'FindAssembly',
 'GetClrType',
 'ListAssemblies',
 'Microsoft',
 'Python',
 'RobotOM',
 'System',
 '_AtExit',
 '__class__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__version__',
 '_extras',
 'clrModule',
 'clrmethod',
 'clrproperty',
 'e__NativeCall',
 'getPreload',
 'setPreload']

In [ ]:
print(dir(rb.CmpntFactory))
rec = ar.RobotOM.IRobotLoadRecordType.IsDefined()

In [ ]:
print([n for n in dir(ar.RobotOM) if "Record" in n])